In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests

# Import the API key
from config import g_key

In [2]:
# Store the CSV created in the WeatherPy project in a data frame
city_data_df = pd.read_csv("weather_data/cities.csv")

city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bambous Virieux,MU,2022-20-09/19/22 15:20:51,-20.3428,57.7575,72.21,78,75,10.36
1,1,Phan Thiet,VN,2022-16-09/19/22 15:16:55,10.9333,108.1000,79.18,81,79,8.46
2,2,Avarua,CK,2022-20-09/19/22 15:20:29,-21.2078,-159.7750,73.45,83,100,2.30
3,3,Kushima,JP,2022-20-09/19/22 15:20:55,31.4583,131.2333,71.20,89,100,9.19
4,4,Mount Isa,AU,2022-20-09/19/22 15:20:55,-20.7333,139.5000,73.17,46,30,0.00


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use the Google API key

gmaps.configure(api_key=g_key)

In [5]:
# # Now create a heat map

# # Assign the locations to an array of lat/long pairs
# locations = [latitude, longitude]

# # Assign the weights variable to some values
# temperatures = # an array of length equal to the locations array length

# # Assign the figure variable to the gmaps.figure() attribute
# fig = gmaps.figure()

# # Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations
# heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# # Add the heatmap layer
# fig.add_layer(heatmap_layer)

# # Call the figure to plot the data
# fig

In [6]:
# Temperature heatmap

# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature
max_temp = city_data_df["Max Temp"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=[max(temp, 0) for temp in max_temp], 
                                 dissipating=False, 
                                 max_intensity=300,
                                 point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Humidity heatmap

# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the humidity
humidity = city_data_df["Humidity"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=300,
                                 point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Cloudiness heatmap

# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the cloudiness
cloudiness = city_data_df["Cloudiness"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=cloudiness, 
                                 dissipating=False, 
                                 max_intensity=300,
                                 point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Wind speed heatmap

# Get the latitude and longitude
locations = city_data_df[["Lat", "Lng"]]

# Get the cloudiness
wind = city_data_df["Wind Speed"]

# Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=wind, 
                                 dissipating=False, 
                                 max_intensity=300,
                                 point_radius=4)

# Add the heatmap layer
fig.add_layer(heat_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask customer for minimum and maximum temperature preferences

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [26]:
# Filter the data set to find the cities that meet the customer's temperature criteria

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Max Temp"] <= max_temp)].dropna()

# Limit the cities to 200
preferred_cities_df = preferred_cities_df.head(200)

In [27]:
preferred_cities_df.count()

City_ID       200
City          200
Country       200
Date          200
Lat           200
Lng           200
Max Temp      200
Humidity      200
Cloudiness    200
Wind Speed    200
dtype: int64

In [28]:
# Create dataframe to store hotel names and city info

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Phan Thiet,VN,79.18,10.9333,108.1000,
10,Merauke,ID,78.98,-8.4667,140.3333,
11,Butaritari,KI,81.68,3.0707,172.7902,
12,Monrovia,LR,80.37,6.3005,-10.7969,
16,Cabo San Lucas,MX,81.52,22.8909,-109.9124,


In [29]:
# Set parameters to search for a hotel

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [36]:
# Iterate through the dataframe

for index, row in hotel_df.iterrows():
    # Get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to a location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get JSON data using our parameters
    hotels = requests.get(base_url, params=params).json()
    
    # Store first hotel name from the results
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [38]:
hotel_df.head(20)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Phan Thiet,VN,79.18,10.9333,108.1000,Hotel Saigon-PT
10,Merauke,ID,78.98,-8.4667,140.3333,
11,Butaritari,KI,81.68,3.0707,172.7902,Isles Sunset Lodge
12,Monrovia,LR,80.37,6.3005,-10.7969,Sunset Inn
16,Cabo San Lucas,MX,81.52,22.8909,-109.9124,Hotel Tesoro Los Cabos
21,Arraial Do Cabo,BR,78.76,-22.9661,-42.0278,Pousada Porto Praia
26,Sao Jose Da Coroa Grande,BR,80.85,-8.8978,-35.1478,Crôa Mares Hotel
34,Buchanan,LR,80.89,5.8808,-10.0467,Kamaneahn Hotel Inc
40,Natal,BR,88.02,-5.7950,-35.2094,Villa Park Hotel | Natal
41,Thinadhoo,MV,83.21,0.5333,72.9333,


In [41]:
# Add a pop-up box for each hotel

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [42]:
# Add a heatmap of temperature for the vacation spots and marker for each city

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
#